In [1]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout

import pandas as pd
import numpy as np

In [7]:
from tensorflow.keras.utils import to_categorical

In [2]:

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
X_train = np.load('/content/X_train.npy', allow_pickle= True)
y_train = np.load('/content/Y_train.npy', allow_pickle= True)

#Remains same regardless of oversampling
X_test = np.load('/content/X_test.npy', allow_pickle= True)
y_test = np.load('/content/Y_test.npy', allow_pickle= True)


print(len(X_train),len(y_train),len(X_test),len(y_test))

1077 1077 462 462


In [17]:
print(X_train.shape)
print(y_train.shape)

(1077, 18)
(1077,)


In [18]:
# Convert target classes to categorical ones
#Y_train = to_categorical(y_train, 2)
#Y_test = to_categorical(y_test, 2)
Y_train= y_train
Y_test = y_test
print(Y_train.shape)

(1077,)


In [19]:
feature_vector_length = 18
num_classes = 1
# Set the input shape
input_shape = (feature_vector_length,)
print(f'Feature shape: {input_shape}')

# Create the model
model = Sequential()
model.add(Dense(350, input_shape=input_shape, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))

Feature shape: (18,)


In [20]:
# Configure the model and start training
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=32, verbose=1, validation_split=0.1)

Epoch 1/10
31/31 [==============================] - 1s 9ms/step - loss: 0.1707 - accuracy: 0.7822 - val_loss: 0.1507 - val_accuracy: 0.8056
Epoch 2/10
31/31 [==============================] - 0s 5ms/step - loss: 0.1503 - accuracy: 0.8070 - val_loss: 0.1462 - val_accuracy: 0.8056
Epoch 3/10
31/31 [==============================] - 0s 5ms/step - loss: 0.1472 - accuracy: 0.8080 - val_loss: 0.1432 - val_accuracy: 0.8056
Epoch 4/10
31/31 [==============================] - 0s 5ms/step - loss: 0.1438 - accuracy: 0.8091 - val_loss: 0.1411 - val_accuracy: 0.8056
Epoch 5/10
31/31 [==============================] - 0s 5ms/step - loss: 0.1437 - accuracy: 0.8132 - val_loss: 0.1411 - val_accuracy: 0.8056
Epoch 6/10
31/31 [==============================] - 0s 5ms/step - loss: 0.1432 - accuracy: 0.8101 - val_loss: 0.1409 - val_accuracy: 0.8056
Epoch 7/10
31/31 [==============================] - 0s 5ms/step - loss: 0.1418 - accuracy: 0.8080 - val_loss: 0.1379 - val_accuracy: 0.8148
Epoch 8/10
31/31 [==

In [21]:
# Test the model after training
test_results = model.evaluate(X_test, Y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

15/15 [==============================] - 0s 3ms/step - loss: 0.1486 - accuracy: 0.8009
Test results - Loss: 0.14864103496074677 - Accuracy: 0.8008658289909363%


In [16]:
!pip install -q -U keras-tuner
import keras_tuner as kt

     |████████████████████████████████| 98 kB 3.1 MB/s 


In [24]:
import tensorflow as tf
import keras

In [48]:
def model_builder(hp):
  model = keras.Sequential()

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=1024, step=32)
  model.add(keras.layers.Dense(units=hp_units, input_shape=input_shape,activation='relu'))
  #Second Hidden Layer
  hp_units_2 = hp.Int('units_2', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units_2, activation='relu'))
  
  # Output Layer
  model.add(Dense(num_classes, activation='sigmoid'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mse',
                metrics=['accuracy'])

  return model

In [49]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=30,
                     factor=3,
                     directory='my_dir',
                     project_name='ann3')

In [50]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [51]:
tuner.search(X_train, Y_train, epochs=100, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}  and the second densely-connected layer is {best_hps.get('units_2')}and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 90 Complete [00h 00m 02s]
val_accuracy: 0.8287037014961243

Best val_accuracy So Far: 0.8287037014961243
Total elapsed time: 00h 02m 52s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 704  and the second densely-connected layer is 32and the optimal learning rate for the optimizer
is 0.01.



In [53]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
27/27 [==============================] - 1s 11ms/step - loss: 0.1770 - accuracy: 0.8014 - val_loss: 0.1423 - val_accuracy: 0.8241
Epoch 2/50
27/27 [==============================] - 0s 7ms/step - loss: 0.1523 - accuracy: 0.8014 - val_loss: 0.1448 - val_accuracy: 0.8009
Epoch 3/50
27/27 [==============================] - 0s 5ms/step - loss: 0.1491 - accuracy: 0.8130 - val_loss: 0.1361 - val_accuracy: 0.8194
Epoch 4/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1446 - accuracy: 0.8118 - val_loss: 0.1347 - val_accuracy: 0.8287
Epoch 5/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1501 - accuracy: 0.8107 - val_loss: 0.1353 - val_accuracy: 0.8194
Epoch 6/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1474 - accuracy: 0.8130 - val_loss: 0.1455 - val_accuracy: 0.8102
Epoch 7/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1453 - accuracy: 0.8153 - val_loss: 0.1389 - val_accuracy: 0.8194
Epoch 8/50
27/27 [=

In [54]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, Y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/4
27/27 [==============================] - 1s 11ms/step - loss: 0.1680 - accuracy: 0.8026 - val_loss: 0.1390 - val_accuracy: 0.8241
Epoch 2/4
27/27 [==============================] - 0s 6ms/step - loss: 0.1549 - accuracy: 0.8084 - val_loss: 0.1391 - val_accuracy: 0.8194
Epoch 3/4
27/27 [==============================] - 0s 5ms/step - loss: 0.1462 - accuracy: 0.8072 - val_loss: 0.1352 - val_accuracy: 0.8194
Epoch 4/4
27/27 [==============================] - 0s 6ms/step - loss: 0.1508 - accuracy: 0.8072 - val_loss: 0.1340 - val_accuracy: 0.8194


In [55]:
eval_result = hypermodel.evaluate(X_test, Y_test)
print("[test loss, test accuracy]:", eval_result)

15/15 [==============================] - 0s 3ms/step - loss: 0.1482 - accuracy: 0.8052
[test loss, test accuracy]: [0.14823348820209503, 0.8051947951316833]


# With Dropout

In [57]:
def model_builder(hp):
  model = keras.Sequential()

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=1024, step=32)
  model.add(keras.layers.Dense(units=hp_units, input_shape=input_shape,activation='relu'))
  #Dropout 1
  hp_float =hp.Float('dropout_1',min_value=0.0,max_value=0.5,default=0.2,step=0.05)
  model.add(Dropout(rate=hp_float))

  #Second Hidden Layer
  hp_units_2 = hp.Int('units_2', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units_2, activation='relu'))
  #Dropout 2
  hp_float_2 =hp.Float('dropout_2',min_value=0.0,max_value=0.5,default=0.25,step=0.05)
  model.add(Dropout(rate=hp_float_2))
  # Output Layer
  model.add(Dense(num_classes, activation='sigmoid'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mse',
                metrics=['accuracy'])

  return model

In [58]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory='my_dir',
                     project_name='ann4DR')

In [59]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)

In [60]:
tuner.search(X_train, Y_train, epochs=100, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}  and the second densely-connected layer is {best_hps.get('units_2')}and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}. Dropout 1 is {best_hps.get('dropout_1')} and Dropout 2 is {best_hps.get('dropout_2')}
""")

Trial 90 Complete [00h 00m 02s]
val_accuracy: 0.8240740895271301

Best val_accuracy So Far: 0.8333333134651184
Total elapsed time: 00h 03m 45s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 960  and the second densely-connected layer is 128and the optimal learning rate for the optimizer
is 0.01. Dropout 1 is 0.5 and Dropout 2 is 0.15000000000000002



In [61]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
27/27 [==============================] - 1s 11ms/step - loss: 0.2021 - accuracy: 0.7991 - val_loss: 0.1759 - val_accuracy: 0.8241
Epoch 2/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1974 - accuracy: 0.8026 - val_loss: 0.1759 - val_accuracy: 0.8241
Epoch 3/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1974 - accuracy: 0.8026 - val_loss: 0.1759 - val_accuracy: 0.8241
Epoch 4/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1974 - accuracy: 0.8026 - val_loss: 0.1759 - val_accuracy: 0.8241
Epoch 5/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1974 - accuracy: 0.8026 - val_loss: 0.1759 - val_accuracy: 0.8241
Epoch 6/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1974 - accuracy: 0.8026 - val_loss: 0.1759 - val_accuracy: 0.8241
Epoch 7/50
27/27 [==============================] - 0s 6ms/step - loss: 0.1974 - accuracy: 0.8026 - val_loss: 0.1759 - val_accuracy: 0.8241
Epoch 8/50
27/27 [=

In [62]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, Y_train, epochs=best_epoch, validation_split=0.2)

27/27 [==============================] - 1s 11ms/step - loss: 0.2022 - accuracy: 0.7979 - val_loss: 0.1759 - val_accuracy: 0.8241


In [63]:
eval_result = hypermodel.evaluate(X_test, Y_test)
print("[test loss, test accuracy]:", eval_result)

15/15 [==============================] - 0s 3ms/step - loss: 0.1926 - accuracy: 0.8074
[test loss, test accuracy]: [0.19264069199562073, 0.8073592782020569]


In [64]:
hypermodel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 960)               18240     
                                                                 
 dropout_4 (Dropout)         (None, 960)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               123008    
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 141,377
Trainable params: 141,377
Non-trainable params: 0
_________________________________________________________________


In [65]:
 y_pred = hypermodel.predict(X_test)

In [68]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [75]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve
 #calc metrics
actual = Y_test
predicted = y_pred
curr_macro_prec = precision_score(actual, predicted,average='weighted')
curr_macro_rec = recall_score(actual, predicted,average='weighted')
curr_macro_f1 = f1_score(actual, predicted,average='weighted')

print('Weighted Precision',curr_macro_prec)
print('Weighted Recall',curr_macro_rec)
print('Weighted F1',curr_macro_f1)

Weighted Precision 0.6518290511797006
Weighted Recall 0.8073593073593074
Weighted F1 0.7213054410659201


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SMOTE OVERSAMPLING

In [76]:
X_train = np.load('/content/X_train_smote.npy', allow_pickle= True)
y_train = np.load('/content/Y_train_smote.npy', allow_pickle= True)

#Remains same regardless of oversampling
X_test = np.load('/content/X_test.npy', allow_pickle= True)
y_test = np.load('/content/Y_test.npy', allow_pickle= True)


print(len(X_train),len(y_train),len(X_test),len(y_test))

Y_train= y_train
Y_test = y_test
print(Y_train.shape)

1738 1738 462 462
(1738,)


In [78]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory='my_dir',
                     project_name='ann4DR_SMOTE')

In [79]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)

In [80]:
tuner.search(X_train, Y_train, epochs=100, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
SMOTE The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}  and the second densely-connected layer is {best_hps.get('units_2')}and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}. Dropout 1 is {best_hps.get('dropout_1')} and Dropout 2 is {best_hps.get('dropout_2')}
""")

Trial 90 Complete [00h 00m 09s]
val_accuracy: 0.8879310488700867

Best val_accuracy So Far: 0.8994252681732178
Total elapsed time: 00h 05m 18s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



SMOTE The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 832  and the second densely-connected layer is 288and the optimal learning rate for the optimizer
is 0.001. Dropout 1 is 0.05 and Dropout 2 is 0.0



In [81]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
44/44 [==============================] - 1s 10ms/step - loss: 0.2242 - accuracy: 0.6309 - val_loss: 0.3169 - val_accuracy: 0.3822
Epoch 2/50
44/44 [==============================] - 0s 5ms/step - loss: 0.2077 - accuracy: 0.6813 - val_loss: 0.4616 - val_accuracy: 0.1868
Epoch 3/50
44/44 [==============================] - 0s 5ms/step - loss: 0.2044 - accuracy: 0.6863 - val_loss: 0.2663 - val_accuracy: 0.5144
Epoch 4/50
44/44 [==============================] - 0s 6ms/step - loss: 0.1928 - accuracy: 0.7094 - val_loss: 0.2132 - val_accuracy: 0.6264
Epoch 5/50
44/44 [==============================] - 0s 6ms/step - loss: 0.1840 - accuracy: 0.7288 - val_loss: 0.2436 - val_accuracy: 0.5460
Epoch 6/50
44/44 [==============================] - 0s 5ms/step - loss: 0.1815 - accuracy: 0.7324 - val_loss: 0.3700 - val_accuracy: 0.3621
Epoch 7/50
44/44 [==============================] - 0s 6ms/step - loss: 0.1816 - accuracy: 0.7345 - val_loss: 0.3211 - val_accuracy: 0.4052
Epoch 8/50
44/44 [=

In [82]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, Y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/39
44/44 [==============================] - 1s 9ms/step - loss: 0.2202 - accuracy: 0.6432 - val_loss: 0.3955 - val_accuracy: 0.2385
Epoch 2/39
44/44 [==============================] - 0s 6ms/step - loss: 0.2055 - accuracy: 0.6928 - val_loss: 0.3012 - val_accuracy: 0.4052
Epoch 3/39
44/44 [==============================] - 0s 5ms/step - loss: 0.1972 - accuracy: 0.7043 - val_loss: 0.2772 - val_accuracy: 0.4770
Epoch 4/39
44/44 [==============================] - 0s 5ms/step - loss: 0.1897 - accuracy: 0.7151 - val_loss: 0.3277 - val_accuracy: 0.3621
Epoch 5/39
44/44 [==============================] - 0s 5ms/step - loss: 0.1915 - accuracy: 0.7094 - val_loss: 0.3431 - val_accuracy: 0.3333
Epoch 6/39
44/44 [==============================] - 0s 6ms/step - loss: 0.1808 - accuracy: 0.7331 - val_loss: 0.2096 - val_accuracy: 0.6609
Epoch 7/39
44/44 [==============================] - 0s 6ms/step - loss: 0.1735 - accuracy: 0.7468 - val_loss: 0.1738 - val_accuracy: 0.7960
Epoch 8/39
44/44 [==

In [83]:
eval_result = hypermodel.evaluate(X_test, Y_test)
print("[test loss, test accuracy]:", eval_result)

15/15 [==============================] - 0s 3ms/step - loss: 0.2319 - accuracy: 0.7013
[test loss, test accuracy]: [0.23186522722244263, 0.701298713684082]


In [84]:
hypermodel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 832)               15808     
                                                                 
 dropout_4 (Dropout)         (None, 832)               0         
                                                                 
 dense_7 (Dense)             (None, 288)               239904    
                                                                 
 dropout_5 (Dropout)         (None, 288)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                 289       
                                                                 
Total params: 256,001
Trainable params: 256,001
Non-trainable params: 0
_________________________________________________________________


In [85]:
 y_pred = hypermodel.predict(X_test)

In [86]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [87]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve
 #calc metrics
actual = Y_test
predicted = y_pred
curr_macro_prec = precision_score(actual, predicted,average='weighted')
curr_macro_rec = recall_score(actual, predicted,average='weighted')
curr_macro_f1 = f1_score(actual, predicted,average='weighted')

print('Weighted Precision',curr_macro_prec)
print('Weighted Recall',curr_macro_rec)
print('Weighted F1',curr_macro_f1)

Weighted Precision 0.7012987012987013
Weighted Recall 0.7012987012987013
Weighted F1 0.7012987012987013


# Borderline SMOTE

In [88]:
X_train = np.load('/content/X_train_borderline_smote.npy', allow_pickle= True)
y_train = np.load('/content/Y_train_borderline_smote.npy', allow_pickle= True)

#Remains same regardless of oversampling
X_test = np.load('/content/X_test.npy', allow_pickle= True)
y_test = np.load('/content/Y_test.npy', allow_pickle= True)


print(len(X_train),len(y_train),len(X_test),len(y_test))

Y_train= y_train
Y_test = y_test
print(Y_train.shape)

1738 1738 462 462
(1738,)


In [89]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory='my_dir',
                     project_name='ann4DR_BL_SMOTE')

In [90]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)

In [91]:
tuner.search(X_train, Y_train, epochs=100, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
SMOTE The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}  and the second densely-connected layer is {best_hps.get('units_2')}and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}. Dropout 1 is {best_hps.get('dropout_1')} and Dropout 2 is {best_hps.get('dropout_2')}
""")

Trial 90 Complete [00h 00m 15s]
val_accuracy: 0.931034505367279

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 05m 03s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



SMOTE The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 960  and the second densely-connected layer is 448and the optimal learning rate for the optimizer
is 0.01. Dropout 1 is 0.0 and Dropout 2 is 0.15000000000000002



In [92]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
44/44 [==============================] - 1s 10ms/step - loss: 0.3742 - accuracy: 0.6252 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 2/50
44/44 [==============================] - 0s 6ms/step - loss: 0.3748 - accuracy: 0.6252 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 3/50
44/44 [==============================] - 0s 5ms/step - loss: 0.3748 - accuracy: 0.6252 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 4/50
44/44 [==============================] - 0s 5ms/step - loss: 0.3748 - accuracy: 0.6252 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 5/50
44/44 [==============================] - 0s 6ms/step - loss: 0.3748 - accuracy: 0.6252 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 6/50
44/44 [==============================] - 0s 5ms/step - loss: 0.3748 - accuracy: 0.6252 - val_loss: 1.0000 - val_accuracy: 0.0000e+00
Epoch 7/50
44/44 [==============================] - 0s 6ms/step - loss: 0.3748 - accuracy: 0.6252 - val_loss: 1.0000 - val_accuracy: 0.

In [93]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, Y_train, epochs=best_epoch, validation_split=0.2)

44/44 [==============================] - 1s 10ms/step - loss: 0.2410 - accuracy: 0.6439 - val_loss: 0.2090 - val_accuracy: 0.6983


In [94]:
eval_result = hypermodel.evaluate(X_test, Y_test)
print("[test loss, test accuracy]:", eval_result)

15/15 [==============================] - 0s 3ms/step - loss: 0.2372 - accuracy: 0.6126
[test loss, test accuracy]: [0.23718969523906708, 0.612554132938385]


In [95]:
hypermodel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 960)               18240     
                                                                 
 dropout_4 (Dropout)         (None, 960)               0         
                                                                 
 dense_7 (Dense)             (None, 448)               430528    
                                                                 
 dropout_5 (Dropout)         (None, 448)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                 449       
                                                                 
Total params: 449,217
Trainable params: 449,217
Non-trainable params: 0
_________________________________________________________________


In [96]:
 y_pred = hypermodel.predict(X_test)

In [97]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [98]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve
 #calc metrics
actual = Y_test
predicted = y_pred
curr_macro_prec = precision_score(actual, predicted,average='weighted')
curr_macro_rec = recall_score(actual, predicted,average='weighted')
curr_macro_f1 = f1_score(actual, predicted,average='weighted')

print('Weighted Precision',curr_macro_prec)
print('Weighted Recall',curr_macro_rec)
print('Weighted F1',curr_macro_f1)

Weighted Precision 0.7528614062704971
Weighted Recall 0.6125541125541125
Weighted F1 0.6529795076335644


# ADASYN

In [99]:
X_train = np.load('/content/X_train_adasyn.npy', allow_pickle= True)
y_train = np.load('/content/Y_train_adasyn.npy', allow_pickle= True)

#Remains same regardless of oversampling
X_test = np.load('/content/X_test.npy', allow_pickle= True)
y_test = np.load('/content/Y_test.npy', allow_pickle= True)


print(len(X_train),len(y_train),len(X_test),len(y_test))

Y_train= y_train
Y_test = y_test
print(Y_train.shape)

1686 1686 462 462
(1686,)


In [101]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory='my_dir',
                     project_name='ann4DR_ADASYN')

In [102]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)

In [103]:
tuner.search(X_train, Y_train, epochs=100, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
SMOTE The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}  and the second densely-connected layer is {best_hps.get('units_2')}and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}. Dropout 1 is {best_hps.get('dropout_1')} and Dropout 2 is {best_hps.get('dropout_2')}
""")

Trial 90 Complete [00h 00m 12s]
val_accuracy: 0.3786982297897339

Best val_accuracy So Far: 0.9437869787216187
Total elapsed time: 00h 05m 07s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



SMOTE The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 96  and the second densely-connected layer is 96and the optimal learning rate for the optimizer
is 0.01. Dropout 1 is 0.30000000000000004 and Dropout 2 is 0.1



In [104]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
43/43 [==============================] - 1s 9ms/step - loss: 0.2287 - accuracy: 0.6343 - val_loss: 0.3399 - val_accuracy: 0.1716
Epoch 2/50
43/43 [==============================] - 0s 6ms/step - loss: 0.2177 - accuracy: 0.6669 - val_loss: 0.4018 - val_accuracy: 0.1953
Epoch 3/50
43/43 [==============================] - 0s 6ms/step - loss: 0.2105 - accuracy: 0.6736 - val_loss: 0.3462 - val_accuracy: 0.1391
Epoch 4/50
43/43 [==============================] - 0s 5ms/step - loss: 0.2081 - accuracy: 0.6684 - val_loss: 0.3923 - val_accuracy: 0.1864
Epoch 5/50
43/43 [==============================] - 0s 5ms/step - loss: 0.2089 - accuracy: 0.6795 - val_loss: 0.2339 - val_accuracy: 0.5740
Epoch 6/50
43/43 [==============================] - 0s 6ms/step - loss: 0.2051 - accuracy: 0.6773 - val_loss: 0.3242 - val_accuracy: 0.4320
Epoch 7/50
43/43 [==============================] - 0s 6ms/step - loss: 0.2059 - accuracy: 0.6840 - val_loss: 0.3635 - val_accuracy: 0.2396
Epoch 8/50
43/43 [==

In [105]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, Y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/49
43/43 [==============================] - 1s 10ms/step - loss: 0.2317 - accuracy: 0.6306 - val_loss: 0.4340 - val_accuracy: 0.0266
Epoch 2/49
43/43 [==============================] - 0s 6ms/step - loss: 0.2193 - accuracy: 0.6588 - val_loss: 0.3042 - val_accuracy: 0.3107
Epoch 3/49
43/43 [==============================] - 0s 6ms/step - loss: 0.2139 - accuracy: 0.6699 - val_loss: 0.3250 - val_accuracy: 0.1124
Epoch 4/49
43/43 [==============================] - 0s 6ms/step - loss: 0.2077 - accuracy: 0.6736 - val_loss: 0.3438 - val_accuracy: 0.2663
Epoch 5/49
43/43 [==============================] - 0s 6ms/step - loss: 0.2071 - accuracy: 0.6758 - val_loss: 0.3614 - val_accuracy: 0.1154
Epoch 6/49
43/43 [==============================] - 0s 6ms/step - loss: 0.2013 - accuracy: 0.6862 - val_loss: 0.4233 - val_accuracy: 0.2219
Epoch 7/49
43/43 [==============================] - 0s 6ms/step - loss: 0.2056 - accuracy: 0.6728 - val_loss: 0.3803 - val_accuracy: 0.2071
Epoch 8/49
43/43 [=

In [106]:
eval_result = hypermodel.evaluate(X_test, Y_test)
print("[test loss, test accuracy]:", eval_result)

15/15 [==============================] - 0s 3ms/step - loss: 0.1920 - accuracy: 0.6948
[test loss, test accuracy]: [0.19196873903274536, 0.6948052048683167]


In [107]:
hypermodel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 96)                1824      
                                                                 
 dropout_4 (Dropout)         (None, 96)                0         
                                                                 
 dense_7 (Dense)             (None, 96)                9312      
                                                                 
 dropout_5 (Dropout)         (None, 96)                0         
                                                                 
 dense_8 (Dense)             (None, 1)                 97        
                                                                 
Total params: 11,233
Trainable params: 11,233
Non-trainable params: 0
_________________________________________________________________


In [108]:
 y_pred = hypermodel.predict(X_test)

In [109]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [110]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve
 #calc metrics
actual = Y_test
predicted = y_pred
curr_macro_prec = precision_score(actual, predicted,average='weighted')
curr_macro_rec = recall_score(actual, predicted,average='weighted')
curr_macro_f1 = f1_score(actual, predicted,average='weighted')

print('Weighted Precision',curr_macro_prec)
print('Weighted Recall',curr_macro_rec)
print('Weighted F1',curr_macro_f1)

Weighted Precision 0.7366894891708786
Weighted Recall 0.6948051948051948
Weighted F1 0.7123680582407628


In [ ]:
!zip -r /content/ann_keras_outs.zip /content/my_dir

In [112]:
from google.colab import files
files.download("/content/ann_keras_outs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>